# Gold層の再集計（Silver全体から更新）

このノートブックでは、Silver層の全データから日別売上サマリーを再集計してGold層を更新します。

## 目的
- **集計データ更新**: 新規データ反映による再計算
- **KPI更新**: ダッシュボード用メトリクスの最新化
- **データ整合性**: 全体データに基づく正確な集計

## 再集計の特徴
- **全件再計算**: Silver層の全データを基に集計
- **MERGE処理**: 既存Gold層データとの効率的な統合
- **パフォーマンス**: 日別集計による適切な粒度

## 処理フロー
1. Silver層全データの読み込み
2. 日別・店舗別売上集計
3. Gold層への MERGE INTO による更新


In [0]:
# 環境変数の定義
catalog_name = "users"
schema_name = "yukiteru_koide"
silver_table = "transactions_enriched"
gold_table = "daily_sales_summary"

# CATALOGとSCHEMAを切り替え
spark.sql(f"USE CATALOG {catalog_name}")
spark.sql(f"USE SCHEMA {schema_name}")


In [0]:
# Silverテーブルを読み込み
df_silver = spark.read.table(f"{catalog_name}.{schema_name}.{silver_table}")

# 日別・店舗別に売上（total_price）を集計
from pyspark.sql.functions import sum as _sum

df_summary = (
    df_silver
    .groupBy("transaction_date", "store_id")
    .agg(_sum("total_price").alias("daily_sales"))
)

# 最初に1回だけテーブルを作成（すでに存在していればこの部分は不要）
df_summary.write.format("delta").mode("overwrite").saveAsTable(f"{catalog_name}.{schema_name}.{gold_table}")

# 今後の更新処理：一時ビューに登録
df_summary.createOrReplaceTempView("summary_updates")

# MERGE INTO による更新・追加処理
spark.sql(f"""
MERGE INTO {catalog_name}.{schema_name}.{gold_table} AS target
USING summary_updates AS source
ON target.transaction_date = source.transaction_date
  AND target.store_id = source.store_id
WHEN MATCHED THEN UPDATE SET
  target.daily_sales = source.daily_sales
WHEN NOT MATCHED THEN INSERT (
  transaction_date, store_id, daily_sales
) VALUES (
  source.transaction_date, source.store_id, source.daily_sales
)
""")
